In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/Hugging Face
!ls

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1OAwQPoxfnPPp90tlzRE1fTmfB377ksLA/Hugging Face
best_vitpose_model		   SegFormer_Huggingface_seg_train.ipynb
datasets			   segformer_trained_weights
DETR_Huggingface_det.ipynb	   segformer_trained_weights_47
markerpensseg			   ViT_Huggingface_classify.ipynb
Markerpens_seg			   vit-multilabel-best
mask2former_Huggingface_seg.ipynb  ViTPose_Huggingface_keypoints.ipynb
mask2formerresults		   YOLOS_Huggingface_det.ipynb
mydrive				   YOLOS_Huggingface_det_wandb.ipynb


In [ ]:
import os

def count_images(image_dir, exts={".jpg", ".jpeg", ".png", ".bmp", ".webp"}):
    image_count = 0
    for filename in os.listdir(image_dir):
        if os.path.splitext(filename.lower())[1] in exts:
            image_count += 1
    print(f"Total images: {image_count}")

count_images("datasets/surgical_instruments/images/train")
count_images("datasets/surgical_instruments/images/valid")

Total images: 2855
Total images: 841


In [ ]:
import json
import os

def convert_to_detection_coco(input_path, output_path):
    with open(input_path, 'r') as f:
        coco_data = json.load(f)

    detection_annotations = []
    for ann in coco_data['annotations']:
        # Remove segmentation-related fields
        det_ann = {
            'id': ann['id'],
            'image_id': ann['image_id'],
            'category_id': ann['category_id'],
            'bbox': ann['bbox'],
            'area': ann['area'],
            'iscrowd': ann.get('iscrowd', 0)
        }
        detection_annotations.append(det_ann)

    detection_coco = {
        'images': coco_data['images'],
        'annotations': detection_annotations,
        'categories': coco_data['categories']
    }

    with open(output_path, 'w') as f:
        json.dump(detection_coco, f)

    print(f"Saved detection JSON to: {output_path}")
    print(f"Images: {len(detection_coco['images'])}, Annotations: {len(detection_coco['annotations'])}")

# Paths
train_input = "datasets/surgical_instruments/annotations/instances_train.json"
valid_input = "datasets/surgical_instruments/annotations/instances_valid.json"
train_output = "datasets/surgical_instruments/annotations/instances_train_det.json"
valid_output = "datasets/surgical_instruments/annotations/instances_valid_det.json"

# Run conversion
convert_to_detection_coco(train_input, train_output)
convert_to_detection_coco(valid_input, valid_output)

Saved detection JSON to: datasets/surgical_instruments/annotations/instances_train_det.json
Images: 2855, Annotations: 4895
Saved detection JSON to: datasets/surgical_instruments/annotations/instances_valid_det.json
Images: 841, Annotations: 1215


In [ ]:
# Check train annotations
with open("datasets/surgical_instruments/annotations/instances_train.json") as f:
    train_data = json.load(f)
    print(f"Train images: {len(train_data['images'])}")
    print(f"Train annotations: {len(train_data['annotations'])}")
    print(f"Categories: {train_data['categories']}")

# Check validation
with open("datasets/surgical_instruments/annotations/instances_valid.json") as f:
    valid_data = json.load(f)
    print(f"\nValid images: {len(valid_data['images'])}")
    print(f"Valid annotations: {len(valid_data['annotations'])}")

Train images: 2855
Train annotations: 4895
Categories: [{'id': 1, 'name': 'Grasper'}, {'id': 2, 'name': 'Harmonic_Ace'}, {'id': 3, 'name': 'Myoma_Screw'}, {'id': 4, 'name': 'Needle_Holder'}, {'id': 5, 'name': 'Suction'}, {'id': 6, 'name': 'Trocar'}]

Valid images: 841
Valid annotations: 1215


In [ ]:
import json
from collections import defaultdict
from pprint import pprint

def check_coco_json(ann_file_path):
    with open(ann_file_path, 'r') as f:
        coco_data = json.load(f)

    # Check top-level keys
    assert 'images' in coco_data, "'images' key not found in JSON"
    assert 'annotations' in coco_data, "'annotations' key not found in JSON"
    assert 'categories' in coco_data, "'categories' key not found in JSON"

    print(f"Total images: {len(coco_data['images'])}")
    print(f"Total annotations: {len(coco_data['annotations'])}")
    print(f"Total categories: {len(coco_data['categories'])}")

    print("\nSample category definitions:")
    pprint(coco_data['categories'])

    # Check required annotation keys (excluding segmentation)
    required_ann_keys = {"id", "image_id", "bbox", "category_id"}
    missing_fields = []

    for i, ann in enumerate(coco_data['annotations']):
        if not required_ann_keys.issubset(ann):
            missing_fields.append(i)

    if missing_fields:
        print(f"⚠️ Missing required fields in {len(missing_fields)} annotations. Examples: {missing_fields[:5]}")
    else:
        print("✅ All annotations have required fields: 'id', 'image_id', 'bbox', 'category_id'")

    # Optional: Check iscrowd and area fields
    optional_fields = ["iscrowd", "area"]
    count_missing = defaultdict(int)

    for ann in coco_data['annotations']:
        for field in optional_fields:
            if field not in ann:
                count_missing[field] += 1

    for field in optional_fields:
        if count_missing[field]:
            print(f"⚠️ Missing '{field}' in {count_missing[field]} annotations")
        else:
            print(f"✅ All annotations have '{field}'")

# Example usage:
check_coco_json("datasets/surgical_instruments/annotations/instances_train_det.json")

Total images: 2855
Total annotations: 4895
Total categories: 6

Sample category definitions:
[{'id': 1, 'name': 'Grasper'},
 {'id': 2, 'name': 'Harmonic_Ace'},
 {'id': 3, 'name': 'Myoma_Screw'},
 {'id': 4, 'name': 'Needle_Holder'},
 {'id': 5, 'name': 'Suction'},
 {'id': 6, 'name': 'Trocar'}]
✅ All annotations have required fields: 'id', 'image_id', 'bbox', 'category_id'
✅ All annotations have 'iscrowd'
✅ All annotations have 'area'


In [ ]:
import json

def check_and_clean_coco_annotations(annotation_file, remove_invalid=False):
    """
    Checks bounding boxes in COCO annotation JSON for:
      - NaN values
      - zero or negative width/height

    Args:
      annotation_file (str): Path to COCO-style JSON annotation file
      remove_invalid (bool): If True, remove invalid annotations from JSON in memory

    Returns:
      invalid_ann_count (int): Number of invalid annotations found
      cleaned_annotations (dict): Cleaned COCO dict (if remove_invalid=True)
    """
    with open(annotation_file, 'r') as f:
        coco = json.load(f)

    invalid_ann_count = 0
    valid_annotations = []

    for ann in coco['annotations']:
        bbox = ann.get('bbox', None)
        if bbox is None:
            print(f"Annotation {ann.get('id', 'unknown')} missing bbox")
            invalid_ann_count += 1
            continue

        # Check for NaN in bbox
        if any([v != v for v in bbox]):  # NaN check: v != v is True only if v is NaN
            print(f"Invalid bbox (NaN) in annotation {ann.get('id', 'unknown')}: {bbox}")
            invalid_ann_count += 1
            continue

        x, y, w, h = bbox
        if w <= 0 or h <= 0:
            print(f"Invalid bbox (zero/negative size) in annotation {ann.get('id', 'unknown')}: {bbox}")
            invalid_ann_count += 1
            continue

        # Passed all checks, keep annotation
        valid_annotations.append(ann)

    if remove_invalid:
        print(f"Removed {invalid_ann_count} invalid annotations.")
        coco['annotations'] = valid_annotations
        return invalid_ann_count, coco
    else:
        print(f"Found {invalid_ann_count} invalid annotations.")
        return invalid_ann_count, None


# Example usage:
annotation_path = "datasets/surgical_instruments/annotations/instances_train_det.json"

invalid_count, cleaned_coco = check_and_clean_coco_annotations(annotation_path, remove_invalid=False)

print(f"Total invalid annotations: {invalid_count}")

# If you want to remove and save cleaned JSON:
if invalid_count > 0:
    _, cleaned = check_and_clean_coco_annotations(annotation_path, remove_invalid=True)
    cleaned_path = annotation_path.replace(".json", "_cleaned.json")
    with open(cleaned_path, 'w') as f_out:
        json.dump(cleaned, f_out)
    print(f"Saved cleaned annotations to {cleaned_path}")

Found 0 invalid annotations.
Total invalid annotations: 0


In [ ]:
!pip install -q transformers datasets torchvision pycocotools
!pip install -q accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
from google.colab import userdata
token = userdata.get('Nyi_token')
login(token=token)

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import json
from transformers import AutoProcessor, AutoModelForObjectDetection, TrainingArguments, Trainer
import torchvision.transforms as T

In [ ]:
class SurgicalInstrumentDetDataset(Dataset):
    def __init__(self, images_dir, annotation_file, transforms=None):
        self.images_dir = images_dir
        self.transforms = transforms

        with open(annotation_file) as f:
            coco = json.load(f)

        self.image_id_to_filename = {img["id"]: img["file_name"] for img in coco["images"]}
        self.id_to_annotations = {}
        for ann in coco["annotations"]:
            img_id = ann["image_id"]
            self.id_to_annotations.setdefault(img_id, []).append(ann)
        self.img_ids = list(self.image_id_to_filename.keys())
        self.categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_path = os.path.join(self.images_dir, self.image_id_to_filename[img_id])
        image = Image.open(img_path).convert("RGB")

        annotations = self.id_to_annotations.get(img_id, [])
        target = {
            "image_id": torch.tensor([img_id]),
            "annotations": annotations
        }

        if self.transforms:
            image = self.transforms(image)

        return image, target

In [ ]:
model_name = "PekingU/rtdetr_r101vd_coco_o365"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForObjectDetection.from_pretrained(model_name)

In [ ]:
class RTDETRProcessedDataset(Dataset):
    def __init__(self, base_dataset, processor):
        self.base_dataset = base_dataset
        self.processor = processor

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        image, target = self.base_dataset[idx]

        # Wrap annotations dict for a single image inside a list
        encoding = self.processor(
            images=image,
            annotations=[target],  # 👈 fix: wrap in list
            return_tensors="pt"
        )

        encoding["pixel_values"] = encoding["pixel_values"].squeeze(0)
        encoding["labels"] = encoding["labels"][0]
        return encoding

In [ ]:
train_dataset = SurgicalInstrumentDetDataset(
    images_dir="datasets/surgical_instruments/images/train",
    annotation_file="datasets/surgical_instruments/annotations/instances_train_det.json"
)

valid_dataset = SurgicalInstrumentDetDataset(
    images_dir="datasets/surgical_instruments/images/valid",
    annotation_file="datasets/surgical_instruments/annotations/instances_valid_det.json"
)

train_processed = RTDETRProcessedDataset(train_dataset, processor)
valid_processed = RTDETRProcessedDataset(valid_dataset, processor)

In [ ]:
class RTDETRDataCollator:
    def __call__(self, features):
        pixel_values = torch.stack([f["pixel_values"] for f in features])
        labels = [f["labels"] for f in features]
        return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/tmp/detr-surgical",     # Temporary checkpoint dir
    overwrite_output_dir=True,

    per_device_train_batch_size=16,       # Adjusted to fit typical Colab GPU memory
    per_device_eval_batch_size=32,
    fp16=True,

    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                   # Keep only 1 checkpoint

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",   # DETR default metric (loss)
    greater_is_better=False,              # Lower loss is better

    logging_strategy="no",                # Disable logging
    report_to="none",                     # No wandb/tensorboard

    num_train_epochs=50,
    dataloader_num_workers=4
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_processed,
    eval_dataset=valid_processed,
    tokenizer=processor,
    data_collator=RTDETRDataCollator(),
)


/tmp/ipython-input-29-3115622598.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()
trainer.save_model("datasets/detr_surgical_best_model50")

Epoch,Training Loss,Validation Loss
1,No log,10.707376
2,No log,9.654958
3,No log,10.200691
4,No log,9.618654
5,No log,8.076127
6,No log,9.036953
7,No log,8.149938
8,No log,8.440506
9,No log,8.385855
10,No log,8.598637


There were missing keys in the checkpoint model loaded: ['class_embed.0.weight', 'class_embed.0.bias', 'class_embed.1.weight', 'class_embed.1.bias', 'class_embed.2.weight', 'class_embed.2.bias', 'class_embed.3.weight', 'class_embed.3.bias', 'class_embed.4.weight', 'class_embed.4.bias', 'class_embed.5.weight', 'class_embed.5.bias', 'bbox_embed.0.layers.0.weight', 'bbox_embed.0.layers.0.bias', 'bbox_embed.0.layers.1.weight', 'bbox_embed.0.layers.1.bias', 'bbox_embed.0.layers.2.weight', 'bbox_embed.0.layers.2.bias', 'bbox_embed.1.layers.0.weight', 'bbox_embed.1.layers.0.bias', 'bbox_embed.1.layers.1.weight', 'bbox_embed.1.layers.1.bias', 'bbox_embed.1.layers.2.weight', 'bbox_embed.1.layers.2.bias', 'bbox_embed.2.layers.0.weight', 'bbox_embed.2.layers.0.bias', 'bbox_embed.2.layers.1.weight', 'bbox_embed.2.layers.1.bias', 'bbox_embed.2.layers.2.weight', 'bbox_embed.2.layers.2.bias', 'bbox_embed.3.layers.0.weight', 'bbox_embed.3.layers.0.bias', 'bbox_embed.3.layers.1.weight', 'bbox_embed.3.l

In [ ]:
trainer.train()
trainer.save_model("datasets/detr_surgical_best_model")

Epoch,Training Loss,Validation Loss
1,No log,11.647306
2,No log,12.251397
3,No log,11.116899
4,No log,11.804318
5,No log,10.595315
6,No log,11.831823
7,No log,10.363395
8,No log,10.424090
9,No log,10.370606
10,No log,11.301422


There were missing keys in the checkpoint model loaded: ['class_embed.0.weight', 'class_embed.0.bias', 'class_embed.1.weight', 'class_embed.1.bias', 'class_embed.2.weight', 'class_embed.2.bias', 'class_embed.3.weight', 'class_embed.3.bias', 'class_embed.4.weight', 'class_embed.4.bias', 'class_embed.5.weight', 'class_embed.5.bias', 'bbox_embed.0.layers.0.weight', 'bbox_embed.0.layers.0.bias', 'bbox_embed.0.layers.1.weight', 'bbox_embed.0.layers.1.bias', 'bbox_embed.0.layers.2.weight', 'bbox_embed.0.layers.2.bias', 'bbox_embed.1.layers.0.weight', 'bbox_embed.1.layers.0.bias', 'bbox_embed.1.layers.1.weight', 'bbox_embed.1.layers.1.bias', 'bbox_embed.1.layers.2.weight', 'bbox_embed.1.layers.2.bias', 'bbox_embed.2.layers.0.weight', 'bbox_embed.2.layers.0.bias', 'bbox_embed.2.layers.1.weight', 'bbox_embed.2.layers.1.bias', 'bbox_embed.2.layers.2.weight', 'bbox_embed.2.layers.2.bias', 'bbox_embed.3.layers.0.weight', 'bbox_embed.3.layers.0.bias', 'bbox_embed.3.layers.1.weight', 'bbox_embed.3.l

In [ ]:
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import os

def run_per_class_eval(model, val_dataset, processor, annotation_file, class_names, device="cuda", threshold=0.1):
    """
    Runs COCO bbox evaluation per class for RTDETR model.

    Args:
        model: RTDETR model.
        val_dataset: Validation dataset; must either have 'base_dataset' attribute returning (PIL image, target)
                     or provide preprocessed tensors under 'pixel_values'.
        processor: RTDETR processor to preprocess images and postprocess outputs.
        annotation_file: Path to COCO-format ground truth annotation JSON.
        class_names: List of class names matching COCO category names in order of category IDs.
                     For example: ['no', 'Grasper', 'Harmonic_Ace', ...]
        device: Device string, e.g., 'cuda' or 'cpu'.
        threshold: Detection score threshold.

    Returns:
        None, prints per-class and overall COCO eval metrics.
    """

    # Load COCO ground truth
    coco_gt = COCO(annotation_file)

    # Build a mapping from category name to category ID (COCO IDs)
    cats = coco_gt.loadCats(coco_gt.getCatIds())
    name_to_catid = {cat["name"]: cat["id"] for cat in cats}

    # Map model class index (based on class_names list) to COCO category ID
    class_idx_to_catid = {}
    for idx, cname in enumerate(class_names):
        if cname in name_to_catid:
            class_idx_to_catid[idx] = name_to_catid[cname]
        else:
            class_idx_to_catid[idx] = None

    # Inverse map: category ID -> class index for evaluation printing
    catid_to_class_idx = {v: k for k, v in class_idx_to_catid.items() if v is not None}

    model.eval()
    model.to(device)

    results = []
    for idx, item in enumerate(tqdm(val_dataset, desc="Evaluating")):
        if hasattr(val_dataset, "base_dataset"):
            pil_img, labels = val_dataset.base_dataset[idx]
            pixel_values = processor(images=pil_img, return_tensors="pt").pixel_values.to(device)
            height, width = pil_img.height, pil_img.width
            image_id = labels.get("image_id", idx) if isinstance(labels, dict) else idx
        else:
            pixel_values = item["pixel_values"].unsqueeze(0).to(device)
            labels = item["labels"]
            height, width = labels["size"].tolist()
            image_id = labels["image_id"].item()

        with torch.no_grad():
            outputs = model(pixel_values)

        processed = processor.post_process_object_detection(
            outputs=outputs,
            target_sizes=torch.tensor([[height, width]], device=device),
            threshold=threshold
        )[0]

        for score, label, box in zip(processed["scores"], processed["labels"], processed["boxes"]):
            class_idx = label.item()
            category_id = class_idx_to_catid.get(class_idx, None)
            if category_id is None:
                continue

            x_min, y_min, x_max, y_max = box.tolist()
            bbox = [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

            results.append({
                "image_id": int(image_id),
                "category_id": int(category_id),
                "bbox": bbox,
                "score": float(score.item())
            })

    if len(results) == 0:
        print("⚠️ No predictions found above threshold. Try lowering threshold or check preprocessing.")
        return

    pred_file = "datasets/rtdetr_predictions.json"
    with open(pred_file, "w") as f:
        json.dump(results, f)

    with open(annotation_file, "r") as f:
        coco_data = json.load(f)
    coco_data.setdefault("info", {"description": "patched"})
    coco_data.setdefault("licenses", [{"id": 0, "name": "none", "url": ""}])
    patched_gt_file = "datasets/rtdetr_patched_gt.json"
    with open(patched_gt_file, "w") as f:
        json.dump(coco_data, f)

    coco_gt_eval = COCO(patched_gt_file)
    coco_dt_eval = coco_gt_eval.loadRes(pred_file)

    coco_eval = COCOeval(coco_gt_eval, coco_dt_eval, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    precisions = coco_eval.eval['precision']
    recalls = coco_eval.eval['recall']

    print("\n📊 Per-class Evaluation (IoU=0.50:0.95):")
    print(f"{'Class':<5} {'Name':<20} {'Precision':>9} {'Recall':>9} {'F1 Score':>9}")
    print("-" * 55)

    per_class_p, per_class_r, per_class_f1 = [], [], []

    for cat_id in coco_eval.params.catIds:
        if cat_id not in catid_to_class_idx:
            continue

        i = catid_to_class_idx[cat_id]
        cat_idx = coco_eval.params.catIds.index(cat_id)

        precision = precisions[:, :, cat_idx, 0, 0]
        recall = recalls[:, cat_idx, 0, 0]

        valid_p = precision[precision > -1]
        valid_r = recall[recall > -1]

        p = np.mean(valid_p) if valid_p.size > 0 else 0.0
        r = np.mean(valid_r) if valid_r.size > 0 else 0.0
        f1 = 2 * p * r / (p + r + 1e-8) if (p + r) > 0 else 0.0

        per_class_p.append(p)
        per_class_r.append(r)
        per_class_f1.append(f1)

        print(f"{i:<5} {class_names[i]:<20} {p:9.4f} {r:9.4f} {f1:9.4f}")

    print("\n🔍 Overall Average:")
    print(f"Precision: {np.mean(per_class_p):.4f}")
    print(f"Recall:    {np.mean(per_class_r):.4f}")
    print(f"F1 Score:  {np.mean(per_class_f1):.4f}")

In [ ]:
# 50 epochs
model_name = "PekingU/rtdetr_r101vd_coco_o365"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForObjectDetection.from_pretrained("datasets/detr_surgical_best_model50")

run_per_class_eval(
    model=model,
    val_dataset=valid_processed,  # RTDETRProcessedDataset
    processor=processor,
    annotation_file="datasets/surgical_instruments/annotations/instances_valid_det.json",
    device="cuda",
    class_names = ['no', 'Grasper', 'Harmonic_Ace', 'Myoma_Screw',
               'Needle_Holder', 'Suction', 'Trocar']
)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Evaluating: 100%|██████████| 841/841 [01:56<00:00,  7.20it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.87s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.780
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.864
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.838
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.578
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.783
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.740
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.828
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

In [ ]:
model_name = "PekingU/rtdetr_r101vd_coco_o365"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForObjectDetection.from_pretrained("datasets/detr_surgical_best_model")

run_per_class_eval(
    model=model,
    val_dataset=valid_processed,  # RTDETRProcessedDataset
    processor=processor,
    annotation_file="datasets/surgical_instruments/annotations/instances_valid_det.json",
    device="cuda",
    class_names = ['no', 'Grasper', 'Harmonic_Ace', 'Myoma_Screw',
               'Needle_Holder', 'Suction', 'Trocar']
)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


Evaluating: 100%|██████████| 841/841 [01:44<00:00,  8.04it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.76s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.718
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.792
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.764
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.428
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.723
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.709
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.778
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet